In [1]:
import duckdb

In [2]:
# working with flight data
flight_sql = "select * from 'https://raw.githubusercontent.com/vivekkhandelwal68/seaborn-data/master/flights.csv'"

In [3]:
# duckdb.sql function provides a relation
flight_rel = duckdb.sql(flight_sql)
type(flight_rel)

duckdb.duckdb.DuckDBPyRelation

In [4]:
# calling show on realtion object print the relation in tabular format
flight_rel.show()

┌───────┬───────────┬────────────┐
│ year  │   month   │ passengers │
│ int64 │  varchar  │   int64    │
├───────┼───────────┼────────────┤
│  1949 │ January   │        112 │
│  1949 │ February  │        118 │
│  1949 │ March     │        132 │
│  1949 │ April     │        129 │
│  1949 │ May       │        121 │
│  1949 │ June      │        135 │
│  1949 │ July      │        148 │
│  1949 │ August    │        148 │
│  1949 │ September │        136 │
│  1949 │ October   │        119 │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│  1960 │ March     │        419 │
│  1960 │ April     │        461 │
│  1960 │ May       │        472 │
│  1960 │ June      │        535 │
│  1960 │ July      │        622 │
│  1960 │ August    │        606 │
│  1960 │ September │        508 │
│  1960 │ October   │        461 │
│  1960 │ November  │        390 │
│  1960 │ December  │        432 │
├───────┴───────────┴────────────┤
│ 144 rows (20 shown

In [5]:
# use of where clause
duckdb.sql("select * from flight_rel where month='July'")

┌───────┬─────────┬────────────┐
│ year  │  month  │ passengers │
│ int64 │ varchar │   int64    │
├───────┼─────────┼────────────┤
│  1949 │ July    │        148 │
│  1950 │ July    │        170 │
│  1951 │ July    │        199 │
│  1952 │ July    │        230 │
│  1953 │ July    │        264 │
│  1954 │ July    │        302 │
│  1955 │ July    │        364 │
│  1956 │ July    │        413 │
│  1957 │ July    │        465 │
│  1958 │ July    │        491 │
│  1959 │ July    │        548 │
│  1960 │ July    │        622 │
├───────┴─────────┴────────────┤
│ 12 rows            3 columns │
└──────────────────────────────┘

In [6]:
# use of group by and having where total passengers count is greater than 200
duckdb.sql("select year, sum(passengers) as total_passengers from flight_rel group by year having sum(passengers) > 200 order by 1")

┌───────┬──────────────────┐
│ year  │ total_passengers │
│ int64 │      int128      │
├───────┼──────────────────┤
│  1949 │             1520 │
│  1950 │             1676 │
│  1951 │             2042 │
│  1952 │             2364 │
│  1953 │             2700 │
│  1954 │             2867 │
│  1955 │             3408 │
│  1956 │             3939 │
│  1957 │             4421 │
│  1958 │             4572 │
│  1959 │             5140 │
│  1960 │             5714 │
├───────┴──────────────────┤
│ 12 rows        2 columns │
└──────────────────────────┘

In [7]:
# working with Pivot operator
pivot_rel = duckdb.sql("SELECT * FROM (PIVOT flight_rel ON month USING sum(passengers) GROUP BY year)")
pivot_rel.show(max_rows=2)

┌───────┬────────┬────────┬──────────┬──────────┬───┬────────┬────────┬────────┬──────────┬─────────┬───────────┐
│ year  │ April  │ August │ December │ February │ … │  June  │ March  │  May   │ November │ October │ September │
│ int64 │ int128 │ int128 │  int128  │  int128  │   │ int128 │ int128 │ int128 │  int128  │ int128  │  int128   │
├───────┼────────┼────────┼──────────┼──────────┼───┼────────┼────────┼────────┼──────────┼─────────┼───────────┤
│  1951 │    163 │    199 │      166 │      150 │ … │    178 │    178 │    172 │      146 │     162 │       184 │
│    ·  │     ·  │     ·  │       ·  │       ·  │ · │     ·  │     ·  │     ·  │       ·  │      ·  │        ·  │
│    ·  │     ·  │     ·  │       ·  │       ·  │ · │     ·  │     ·  │     ·  │       ·  │      ·  │        ·  │
│    ·  │     ·  │     ·  │       ·  │       ·  │ · │     ·  │     ·  │     ·  │       ·  │      ·  │        ·  │
│  1949 │    129 │    148 │      118 │      118 │ … │    135 │    132 │    121 │      10

In [8]:
# stroing file as parquet
pivot_rel.write_parquet("flight_pivot.parquet")

In [9]:
# reading parquet into relation(in same manner read_csv, read_parquet, read_json can be used for creating realtion)
paruet_rel = duckdb.read_parquet("flight_pivot.parquet")
paruet_rel

┌───────┬────────┬────────┬──────────┬──────────┬───┬────────┬────────┬────────┬──────────┬─────────┬───────────┐
│ year  │ April  │ August │ December │ February │ … │  June  │ March  │  May   │ November │ October │ September │
│ int64 │ double │ double │  double  │  double  │   │ double │ double │ double │  double  │ double  │  double   │
├───────┼────────┼────────┼──────────┼──────────┼───┼────────┼────────┼────────┼──────────┼─────────┼───────────┤
│  1951 │  163.0 │  199.0 │    166.0 │    150.0 │ … │  178.0 │  178.0 │  172.0 │    146.0 │   162.0 │     184.0 │
│  1960 │  461.0 │  606.0 │    432.0 │    391.0 │ … │  535.0 │  419.0 │  472.0 │    390.0 │   461.0 │     508.0 │
│  1954 │  227.0 │  293.0 │    229.0 │    188.0 │ … │  264.0 │  235.0 │  234.0 │    203.0 │   229.0 │     259.0 │
│  1958 │  348.0 │  505.0 │    337.0 │    318.0 │ … │  435.0 │  362.0 │  363.0 │    310.0 │   359.0 │     404.0 │
│  1955 │  269.0 │  347.0 │    278.0 │    233.0 │ … │  315.0 │  267.0 │  270.0 │    237.

In [10]:
# generating row numbers
duckdb.sql("select * , row_number() over( partition by year order by year desc) as rn from flight_rel")

┌───────┬───────────┬────────────┬───────┐
│ year  │   month   │ passengers │  rn   │
│ int64 │  varchar  │   int64    │ int64 │
├───────┼───────────┼────────────┼───────┤
│  1953 │ July      │        264 │     1 │
│  1953 │ August    │        272 │     2 │
│  1953 │ September │        237 │     3 │
│  1953 │ October   │        211 │     4 │
│  1953 │ November  │        180 │     5 │
│  1953 │ December  │        201 │     6 │
│  1953 │ January   │        196 │     7 │
│  1953 │ February  │        196 │     8 │
│  1953 │ March     │        236 │     9 │
│  1953 │ April     │        235 │    10 │
│    ·  │   ·       │         ·  │     · │
│    ·  │   ·       │         ·  │     · │
│    ·  │   ·       │         ·  │     · │
│  1960 │ September │        508 │     9 │
│  1960 │ October   │        461 │    10 │
│  1960 │ November  │        390 │    11 │
│  1960 │ December  │        432 │    12 │
│  1956 │ July      │        413 │     7 │
│  1956 │ August    │        405 │     8 │
│  1956 │ S

In [11]:
# reading json into a realtion
movie_rel = duckdb.read_json("https://raw.githubusercontent.com/vivekkhandelwal68/imdb-scrap/master/output/nolanDB.json")
movie_rel.show(max_rows=20)

┌────────────────────┬──────────┬───────────────────┬───┬──────────────────────┬──────────────────────┬──────────┐
│      @context      │  @type   │        url        │ … │        review        │   aggregateRating    │ duration │
│      varchar       │ varchar  │      varchar      │   │ struct("@type" var…  │ struct("@type" var…  │ varchar  │
├────────────────────┼──────────┼───────────────────┼───┼──────────────────────┼──────────────────────┼──────────┤
│ https://schema.org │ Movie    │ /title/tt0209144/ │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT1H53M  │
│ https://schema.org │ Movie    │ /title/tt6723592/ │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT2H30M  │
│ https://schema.org │ Movie    │ /title/tt5013056/ │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT1H46M  │
│ https://schema.org │ Movie    │ /title/tt0816692/ │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT2H49M  │
│ https://schema.org │ Movie    │ /title/tt0770828/ │ … │ {'@type': Review, …  │

In [12]:
# Showing all relation columns
movie_rel.columns

['@context',
 '@type',
 'url',
 'name',
 'image',
 'contentRating',
 'genre',
 'actor',
 'director',
 'creator',
 'trailer',
 'datePublished',
 'description',
 'keywords',
 'review',
 'aggregateRating',
 'duration']

In [13]:
# showing all relation columns datatypes
movie_rel.dtypes

[VARCHAR,
 VARCHAR,
 VARCHAR,
 VARCHAR,
 VARCHAR,
 VARCHAR,
 VARCHAR[],
 STRUCT("@type" VARCHAR, url VARCHAR, "name" VARCHAR)[],
 STRUCT("@type" VARCHAR, url VARCHAR, "name" VARCHAR)[],
 STRUCT("@type" VARCHAR, url VARCHAR, "name" VARCHAR)[],
 STRUCT("@type" VARCHAR, "name" VARCHAR, embedUrl VARCHAR, thumbnail STRUCT("@type" VARCHAR, contentUrl VARCHAR), thumbnailUrl VARCHAR, description VARCHAR),
 DATE,
 VARCHAR,
 VARCHAR,
 STRUCT("@type" VARCHAR, itemReviewed STRUCT("@type" VARCHAR, url VARCHAR), author STRUCT("@type" VARCHAR, "name" VARCHAR), dateCreated DATE, inLanguage VARCHAR, "name" VARCHAR, reviewBody VARCHAR, reviewRating STRUCT("@type" VARCHAR, worstRating BIGINT, bestRating BIGINT, ratingValue BIGINT)),
 STRUCT("@type" VARCHAR, ratingCount BIGINT, bestRating BIGINT, worstRating BIGINT, ratingValue DOUBLE),
 VARCHAR]

In [14]:
# describe relation
movie_rel.describe()

┌─────────┬────────────────────┬───────────┬───┬──────────────────────┬──────────────────────┬──────────┐
│  aggr   │      @context      │   @type   │ … │        review        │   aggregateRating    │ duration │
│ varchar │      varchar       │  varchar  │   │       varchar        │       varchar        │ varchar  │
├─────────┼────────────────────┼───────────┼───┼──────────────────────┼──────────────────────┼──────────┤
│ count   │ 144                │ 144       │ … │ 72                   │ 134                  │ 89       │
│ mean    │ NULL               │ NULL      │ … │ NULL                 │ NULL                 │ NULL     │
│ stddev  │ NULL               │ NULL      │ … │ NULL                 │ NULL                 │ NULL     │
│ min     │ https://schema.org │ Movie     │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT12M    │
│ max     │ https://schema.org │ VideoGame │ … │ {'@type': Review, …  │ {'@type': Aggregat…  │ PT9M     │
│ median  │ NULL               │ NULL      │ …

In [15]:
## creating connection to new database
con = duckdb.connect("flight.db")
con.sql("CREATE OR REPLACE TABLE flight_tbl as select * from flight_rel")
con.sql("SELECT * FROM flight_tbl").show()

┌───────┬───────────┬────────────┐
│ year  │   month   │ passengers │
│ int64 │  varchar  │   int64    │
├───────┼───────────┼────────────┤
│  1949 │ January   │        112 │
│  1949 │ February  │        118 │
│  1949 │ March     │        132 │
│  1949 │ April     │        129 │
│  1949 │ May       │        121 │
│  1949 │ June      │        135 │
│  1949 │ July      │        148 │
│  1949 │ August    │        148 │
│  1949 │ September │        136 │
│  1949 │ October   │        119 │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│  1960 │ March     │        419 │
│  1960 │ April     │        461 │
│  1960 │ May       │        472 │
│  1960 │ June      │        535 │
│  1960 │ July      │        622 │
│  1960 │ August    │        606 │
│  1960 │ September │        508 │
│  1960 │ October   │        461 │
│  1960 │ November  │        390 │
│  1960 │ December  │        432 │
├───────┴───────────┴────────────┤
│ 144 rows (20 shown